# QSeaBattle – Quick Start Guide

This notebook demonstrates the core workflow of **QSeaBattle**: defining a game layout, selecting players, running tournaments, and evaluating performance.



## 1. Game layout

The `GameLayout` defines the *problem instance*:

- `field_size` determines the number of battlefield cells (`n2 = field_size**2`).
- `comms_size` determines how many bits Player A may communicate to Player B.

Most players assume uniform random inputs and operate on flattened binary vectors.


## 2. Players and assistance models

Players are always created via a **factory** (e.g. `SimplePlayers`, `MajorityPlayers`,
`PRAssistedPlayers`, `NeuralNetPlayers`).

The factory:
- Validates compatibility with the layout.
- Constructs Player A and Player B.
- Owns any shared resources (classical, PR-assisted, or learned).

You can switch strategies by changing only the factory class.


## 3. Running a tournament

A `Tournament` repeatedly samples random game instances and lets the players interact
with the environment.

The output is a `TournamentLog` containing:
- Win rate
- Per-game outcomes
- Optional player log-probabilities (if supported)

This separation makes it easy to benchmark different player types under identical conditions.


## 4. Interpreting results

When comparing strategies, focus on:

- Relative performance across layouts.
- Sensitivity to noise or adversarial parameters.
- Gaps between classical, PR-assisted, and learned strategies.

The goal is not a single "best" player, but understanding *why* certain resources
provide an advantage.


In [1]:
import sys
import pandas as pd
sys.path.append("../src")
import Q_Sea_Battle as QSB


In [2]:
field_sizes = [4,8,16,32]
number_of_games_in_tournament = 100 #10000
channel_noise_levels = [0.0, 0.1, 0.3, 0.5]



In [3]:
results_list = []
for field_size in field_sizes:
    for noise_level in channel_noise_levels:
        # Layout: variable field size, 1-bit communication, fixed number of games
        layout = QSB.GameLayout(
            field_size=field_size,
            comms_size=1,
            number_of_games_in_tournament=number_of_games_in_tournament,
            channel_noise=noise_level
        )

        env = QSB.GameEnv(layout)
        players = QSB.Players(layout)
        tournament = QSB.Tournament(env, players, layout)

        log = tournament.tournament()
        mean_reward, std_err = log.outcome()

        # Store results in list
        results_list.append({
            'player_type': 'base',
            'field_size': field_size,
            'noise_level': noise_level,
            'performance': mean_reward,
            '95p error +/-': 1.96 * std_err,
            'reference': 0.5,
            'in_interval': (mean_reward - 1.96 * std_err <= 0.5 <= mean_reward + 1.96 * std_err)
        })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT
player_type  field_size  noise_level  performance  95p error +/-  reference  in_interval
       base           4       0.0000       0.4300         0.0975     0.5000         True
       base           4       0.1000       0.4600         0.0982     0.5000         True
       base           4       0.3000       0.4400         0.0978     0.5000         True
       base           4       0.5000       0.5600         0.0978     0.5000         True
       base           8       0.0000       0.4500         0.0980     0.5000         True
       base           8       0.1000       0.4300         0.0975     0.5000         True
       base           8       0.3000       0.5200         0.0984     0.5000         True
       base           8       0.5000       0.4800         0.0984     0.5000         True
       base          16       0.0000       0.4200         0.0972     0.5000         True
       base          16       

In [4]:
results_list = []
for field_size in field_sizes:
    for noise_level in channel_noise_levels:
        # Layout: variable field size, 1-bit communication, fixed number of games
        layout = QSB.GameLayout(
            field_size=field_size,
            comms_size=1,
            number_of_games_in_tournament=number_of_games_in_tournament,
            channel_noise=noise_level
        )

        env = QSB.GameEnv(layout)
        players = QSB.SimplePlayers(layout)
        tournament = QSB.Tournament(env, players, layout)

        log = tournament.tournament()
        mean_reward, std_err = log.outcome()

        ref = QSB.expected_win_rate_simple(field_size = field_size, 
                                           comms_size=1,
                                           channel_noise=noise_level)

        # Store results in list
        results_list.append({
            'player_type': 'simple',
            'field_size': field_size,
            'noise_level': noise_level,
            'performance': mean_reward,
            '95p error +/-': 1.96 * std_err,
            'reference': ref,
            'in_interval': (mean_reward - 1.96 * std_err <= ref <= mean_reward + 1.96 * std_err)
        })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT
player_type  field_size  noise_level  performance  95p error +/-  reference  in_interval
     simple           4       0.0000       0.5000         0.0985     0.5312         True
     simple           4       0.1000       0.5400         0.0982     0.5250         True
     simple           4       0.3000       0.4900         0.0985     0.5125         True
     simple           4       0.5000       0.5200         0.0984     0.5000         True
     simple           8       0.0000       0.5900         0.0969     0.5078         True
     simple           8       0.1000       0.4400         0.0978     0.5062         True
     simple           8       0.3000       0.5200         0.0984     0.5031         True
     simple           8       0.5000       0.4100         0.0969     0.5000         True
     simple          16       0.0000       0.5200         0.0984     0.5020         True
     simple          16       

In [5]:
results_list = []
for field_size in field_sizes:
    for noise_level in channel_noise_levels:
        # Layout: variable field size, 1-bit communication, fixed number of games
        layout = QSB.GameLayout(
            field_size=field_size,
            comms_size=1,
            number_of_games_in_tournament=number_of_games_in_tournament,
            channel_noise=noise_level
        )

        env = QSB.GameEnv(layout)
        players = QSB.MajorityPlayers(layout)
        tournament = QSB.Tournament(env, players, layout)

        log = tournament.tournament()
        mean_reward, std_err = log.outcome()

        ref = QSB.expected_win_rate_majority(field_size = field_size, 
                                           comms_size=1,
                                           channel_noise=noise_level)

        # Store results in list
        results_list.append({
            'player_type': 'majority',
            'field_size': field_size,
            'noise_level': noise_level,
            'performance': mean_reward,
            '95p error +/-': 1.96 * std_err,
            'reference': ref,
            'in_interval': (mean_reward - 1.96 * std_err <= ref <= mean_reward + 1.96 * std_err)
        })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT
player_type  field_size  noise_level  performance  95p error +/-  reference  in_interval
   majority           4       0.0000       0.6500         0.0940     0.5982         True
   majority           4       0.1000       0.4800         0.0984     0.5786        False
   majority           4       0.3000       0.6100         0.0961     0.5393         True
   majority           4       0.5000       0.5500         0.0980     0.5000         True
   majority           8       0.0000       0.5300         0.0983     0.5497         True
   majority           8       0.1000       0.5400         0.0982     0.5397         True
   majority           8       0.3000       0.4500         0.0980     0.5199         True
   majority           8       0.5000       0.4600         0.0982     0.5000         True
   majority          16       0.0000       0.4900         0.0985     0.5249         True
   majority          16       

In [6]:
results_list = []
p_high = 1.0
for field_size in field_sizes:
    for noise_level in channel_noise_levels:
        # Layout: variable field size, 1-bit communication, fixed number of games
        layout = QSB.GameLayout(
            field_size=field_size,
            comms_size=1,
            number_of_games_in_tournament=number_of_games_in_tournament,
            channel_noise=noise_level
        )

        env = QSB.GameEnv(layout)
        players = QSB.PRAssistedPlayers(game_layout = layout, p_high = p_high)
        tournament = QSB.Tournament(env, players, layout)

        log = tournament.tournament()
        mean_reward, std_err = log.outcome()

        ref = QSB.expected_win_rate_assisted(field_size = field_size, 
                                           comms_size=1,
                                           channel_noise=noise_level,
                                           p_high=p_high)

        # Store results in list
        results_list.append({
            'player_type': 'assisted/P_high= '+str(p_high),
            'field_size': field_size,
            'noise_level': noise_level,
            'performance': mean_reward,
            '95p error +/-': 1.96 * std_err,
            'reference': ref,
            'in_interval': (mean_reward - 1.96 * std_err <= ref <= mean_reward + 1.96 * std_err)
        })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT.")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT.
         player_type  field_size  noise_level  performance  95p error +/-  reference  in_interval
assisted/P_high= 1.0           4       0.0000       1.0000         0.0000     1.0000         True
assisted/P_high= 1.0           4       0.1000       0.8800         0.0640     0.9000         True
assisted/P_high= 1.0           4       0.3000       0.6500         0.0940     0.7000         True
assisted/P_high= 1.0           4       0.5000       0.5100         0.0985     0.5000         True
assisted/P_high= 1.0           8       0.0000       1.0000         0.0000     1.0000         True
assisted/P_high= 1.0           8       0.1000       0.8800         0.0640     0.9000         True
assisted/P_high= 1.0           8       0.3000       0.6700         0.0926     0.7000         True
assisted/P_high= 1.0           8       0.5000       0.5400         0.0982     0.5000         True
assisted/P_high= 1.0          16     

In [7]:
results_list = []
p_high = 0.85
for field_size in field_sizes:
    for noise_level in channel_noise_levels:
        # Layout: variable field size, 1-bit communication, fixed number of games
        layout = QSB.GameLayout(
            field_size=field_size,
            comms_size=1,
            number_of_games_in_tournament=number_of_games_in_tournament,
            channel_noise=noise_level
        )

        env = QSB.GameEnv(layout)
        players = QSB.PRAssistedPlayers(game_layout = layout, p_high = p_high)
        tournament = QSB.Tournament(env, players, layout)

        log = tournament.tournament()
        mean_reward, std_err = log.outcome()

        ref = QSB.expected_win_rate_assisted(field_size = field_size, 
                                           comms_size=1,
                                           channel_noise=noise_level,
                                           p_high=p_high)

        # Store results in list
        results_list.append({
            'player_type': 'assisted/P_high= '+str(p_high),
            'field_size': field_size,
            'noise_level': noise_level,
            'performance': mean_reward,
            '95p error +/-': 1.96 * std_err,
            'reference': ref,
            'in_interval': (mean_reward - 1.96 * std_err <= ref <= mean_reward + 1.96 * std_err)
        })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT.")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT.
          player_type  field_size  noise_level  performance  95p error +/-  reference  in_interval
assisted/P_high= 0.85           4       0.0000       0.5800         0.0972     0.6200         True
assisted/P_high= 0.85           4       0.1000       0.5900         0.0969     0.5960         True
assisted/P_high= 0.85           4       0.3000       0.6800         0.0919     0.5480        False
assisted/P_high= 0.85           4       0.5000       0.5100         0.0985     0.5000         True
assisted/P_high= 0.85           8       0.0000       0.5600         0.0978     0.5588         True
assisted/P_high= 0.85           8       0.1000       0.5200         0.0984     0.5471         True
assisted/P_high= 0.85           8       0.3000       0.5600         0.0978     0.5235         True
assisted/P_high= 0.85           8       0.5000       0.4700         0.0983     0.5000         True
assisted/P_high= 0.85       

In [8]:
results_list = []
field_size = 64
noise_level = 0.45
p_high_values = [float(n/100) for n in range(75,95,1)]
for p_high in p_high_values:
    # Layout: variable field size, 1-bit communication, fixed number of games
    layout = QSB.GameLayout(
        field_size=field_size,
        comms_size=1,
        number_of_games_in_tournament=number_of_games_in_tournament,
        channel_noise=noise_level
    )

    env = QSB.GameEnv(layout)
    players = QSB.PRAssistedPlayers(game_layout = layout, p_high = p_high)
    tournament = QSB.Tournament(env, players, layout)

    log = tournament.tournament()
    mean_reward, std_err = log.outcome()

    ref = QSB.expected_win_rate_assisted(field_size = field_size, 
                                        comms_size=1,
                                        channel_noise=noise_level,
                                        p_high=p_high)
    
    ic_bound = QSB.limit_from_mutual_information(field_size=field_size,
                                                    comms_size=1,
                                                    channel_noise=noise_level)

    # Store results in list
    results_list.append({
        'player_type': 'assisted/P_high= '+str(p_high),
        'field_size': field_size,
        'noise_level': noise_level,
        'performance': mean_reward,
        'reference': ref,
        'information_constraint': ic_bound,
        'in_interval': (mean_reward - 1.96 * std_err <= ref <= mean_reward + 1.96 * std_err)
    })

# Create DataFrame from collected results
results_df = pd.DataFrame(results_list)

print("Tournament simulations completed.")
print("\n" + "="*100)
print(f"RESULTS SUMMARY FOR {number_of_games_in_tournament} GAMES PER TOURNAMENT.")
print("="*100)
print(results_df.to_string(index=False, float_format='%.4f'))

print("="*100)

Tournament simulations completed.

RESULTS SUMMARY FOR 100 GAMES PER TOURNAMENT.
          player_type  field_size  noise_level  performance  reference  information_constraint  in_interval
assisted/P_high= 0.75          64       0.4500       0.4200     0.5000                  0.5008         True
assisted/P_high= 0.76          64       0.4500       0.4500     0.5000                  0.5008         True
assisted/P_high= 0.77          64       0.4500       0.4900     0.5000                  0.5008         True
assisted/P_high= 0.78          64       0.4500       0.4900     0.5000                  0.5008         True
assisted/P_high= 0.79          64       0.4500       0.5400     0.5001                  0.5008         True
 assisted/P_high= 0.8          64       0.4500       0.5600     0.5001                  0.5008         True
assisted/P_high= 0.81          64       0.4500       0.5100     0.5002                  0.5008         True
assisted/P_high= 0.82          64       0.4500       0.